# 演習問題　12.2

ある中学校1年生の男子10人の身長$x_1 (\mathrm{cm})$・胸囲 $x_2 (\mathrm{cm})$・体重 $y (\mathrm{kg})$を測定し下記のデータを得た。

|中学生|1|2|3|4|5|6|7|8|9|10|
|:-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|x_1 (cm)|147|160|161|163|155|154|170|171|157|156|
|x_2 (cm)|68|75|77|86|73|74|75|80|74|78|
|y (kg)|37|54|49|64|48|44|49|58|42|52|

胸囲$x_2$の影響を取り除いたときの体重$y$と身長$x_1$の偏相関$r_{01 \cdot (2)}$を求め、体重$y$と身長$x_1$の単相関$x_{01}$と比較せよ。

# 解答　12.2

###次の3つのアプローチを用いて、偏相関係数をそれぞれ算出。
1. 共分散行列の逆行列の要素を用いて計算（教科書での計算方法）
2. 観測値$y$と説明変数$x_1$から、他の説明変数の影響を取り除いたそれぞれの残差の相関係数を計算（教科書での偏相関係数の定義）
3. (単)相関係数を用いて計算（略解に出ている式、他の多くの統計学の本で公式として用いられている）  

### 1. 共分散行列の逆行列の要素を用いて計算

体重の観測値ベクトルを$y$、身長と胸囲のベクトルをそれぞれ$x_1,x_2$とし、デザイン行列を身長、胸囲の順で$X= [x_1,x_2]$とする。  
(教科書とは異なり、n×1の縦ベクトルで扱う。)

In [4]:
import math
import numpy as np

y = np.matrix([ 37,  54,  49,  64,  48,  44,  49,  58,  42,  52]).T
x_1 = np.matrix([147, 160, 161, 163, 155, 154, 170, 171, 157, 156]).T
x_2 = np.matrix([ 68,  75,  77,  86,  73,  74,  75,  80,  74,  78]).T
X = np.column_stack((x_1,x_2))

In [5]:
X #デザイン行列

matrix([[147,  68],
        [160,  75],
        [161,  77],
        [163,  86],
        [155,  73],
        [154,  74],
        [170,  75],
        [171,  80],
        [157,  74],
        [156,  78]])

標本共分散行列と変数行列の関係を簡潔に表現するため、変数の行列を中心化しておく。 
$y$と$X$を中心化したものを、それぞれ$y_c$,$X_c$とおく。  
以下では、中心化を行うオペレータ$(I-(1/n)jj')$を用い、
$y_c= (I-(1/n)jj')y,X_c = (I-(1/n)jj')X$を計算。  
ここで、$n$は観測数、$I$は単位行列、$j=(1,1, \cdots,1)'$。$'$は転置を表す記号。

In [6]:
n = len(y) #観測数 10

I = np.matrix(np.identity(n))#10×10単位行列 
J = np.matrix(np.ones((n,n))) # J = jj'　要素がすべて1の正方行列（10×10）

y_c = (I-(1/n)*J)*y #中心化したy
X_c = (I-(1/n)*J)*X #中心化したX

In [7]:
y_c

matrix([[-12.7],
        [  4.3],
        [ -0.7],
        [ 14.3],
        [ -1.7],
        [ -5.7],
        [ -0.7],
        [  8.3],
        [ -7.7],
        [  2.3]])

In [8]:
X_c

matrix([[-12.4,  -8. ],
        [  0.6,  -1. ],
        [  1.6,   1. ],
        [  3.6,  10. ],
        [ -4.4,  -3. ],
        [ -5.4,  -2. ],
        [ 10.6,  -1. ],
        [ 11.6,   4. ],
        [ -2.4,  -2. ],
        [ -3.4,   2. ]])

説明変数$x_j,x_l(j,l=1,2)$の標本共分散を$s_{jl}$とし、標本共分散行列を$S_{xx}$とする。$S_{xx}=(1/n)X_c'X_c$より、

In [9]:
S_xx = (1/n) * X_c.T *X_c
S_xx

matrix([[ 48.24,  19.4 ],
        [ 19.4 ,  20.4 ]])

説明変数$x_j$と観測値yの標本共分散を$s_{j0}$、そのベクトルを$S_{xy}$とする。$S_{xy}=(1/n)X_c'y_c$より、

In [10]:
S_xy = (1/n) * X_c.T * y_c
S_xy

matrix([[ 34.82],
        [ 31.  ]])

観測値$y$の(共)分散を$s_{00}$とする。

In [11]:
s_00 = (1/n) * y_c.T * y_c

ここで全データの標本共分散行列を$S$と表す。$\begin{equation}S=\begin{pmatrix}s_{00} & S_{xy}'\\  S_{xy} & S_{xx} \end{pmatrix} \end{equation}$.

In [12]:
S = np.bmat([[s_00,S_xy.T],[S_xy,S_xx]])

$S^{-1}$を計算。(コードではS_invで表記）

In [13]:
S_inv = np.linalg.inv(S)
S_inv

matrix([[ 0.13614503, -0.02440205, -0.18368118],
        [-0.02440205,  0.03794093,  0.00100047],
        [-0.18368118,  0.00100047,  0.32719154]])

####偏相関係数$r_{01\cdot(2)}$を求める。

$S^{-1}$の$ij$要素を$s^{ij}$で表す。（コードではs_inv_ij）。  
$y,x_1$の偏相関係数$r_{01\cdot(2)}$は、$-s^{01}/\sqrt{s^{00}s^{11}}$で表せることより(教科書p.215)、

In [14]:
s_inv_00 = S_inv[0,0]
s_inv_01 = S_inv[0,1]
s_inv_11 = S_inv[1,1]

r012 = -s_inv_01/math.sqrt(s_inv_00*s_inv_11)
print("偏相関係数")
r012


偏相関係数


0.33952479611309916

####単相関係数$r_{01}$を求める。

共分散行列$S$の$ij$要素を$s_{ij}$とすれば、$y$と$x_1$の（単）相関係数$r_{01}$は$s_{01}/\sqrt{s_{00}s_{11}}$であるから、

In [15]:
s_00 = S[0,0]
s_01 = S[0,1]
s_11 = S[1,1]

r_01 = s_01/math.sqrt(s_00*s_11)
print("単相関係数")
r_01

単相関係数


0.67348983010735031

####結果の考察

$0$ < $r_{01(2)}$ < $r_{01}$となることから、身長と体重の間には正の相関があるが、胸囲の大きさの影響を取り除くとその関係は弱まる。
すなわち、胸囲（横幅？）を一定とすれば、身長の高さ（縦？）が体重に与える影響はより限定的になる。

###2. 残差の相関係数を計算  
変数等の表記は1.と共通

$y$を$x_2$に回帰したものを$\hat{\hat{y}}$とし、その残差を$e_0$とする。$x_1$を$x_2$に回帰したものを$\hat{\hat{x_1}}$とし、その残差を$e_1$とする（教科書 p.213）。これらの単相関係数$Corr(e_0,e_1)$が、教科書でのそもそもの偏相関係数の定義。

$\hat{\hat{y_c}} = \hat{\hat{\theta_c}}x_{2c}, \hspace{15pt} e_0=y_c - \hat{\hat{y_c}}$  
$\hat{\hat{x_{1c}}} = \hat{\hat{\alpha_c}}x_{2c}, \hspace{15pt} e_1=x_{1c} - \hat{\hat{x_c}}$

$e_0$を求める。$\ e_0 = [I-x_2(x_2'x_2)^{-1}x_2']y_c\ $

In [16]:
x_2c = (I-(1/n)*J)*x_2 #中心化
xx_2c = x_2c.T*x_2c #情報行列
xx_2c_inv = np.linalg.inv(xx_2c) #情報行列の逆行列
e_0 = (I-x_2c*xx_2c_inv*x_2c.T)*y_c # e = [I-X(X'X)^ X']y
print("残差e_0")
e_0

残差e_0


matrix([[-0.54313725],
        [ 5.81960784],
        [-2.21960784],
        [-0.89607843],
        [ 2.85882353],
        [-2.66078431],
        [ 0.81960784],
        [ 2.22156863],
        [-4.66078431],
        [-0.73921569]])

同様に$\ e_1$を求める。

In [17]:
x_1c = (I-(1/n)*J)*x_1 #中心化
xx_1c = x_1c.T*x_1c #情報行列 X'X
xx_1c_inv = np.linalg.inv(xx_1c) #情報行列の逆行列
e_1 = (I-x_2c*xx_2c_inv*x_2c.T)*x_1c #X(X'X)^-1 X'y
e_1

matrix([[ -4.79215686],
        [  1.55098039],
        [  0.64901961],
        [ -5.90980392],
        [ -1.54705882],
        [ -3.49803922],
        [ 11.55098039],
        [  7.79607843],
        [ -0.49803922],
        [ -5.30196078]])

$e_0$と$e_1$の相関係数(相関行列)を求める。

In [18]:
e_0 = np.array(e_0).T #arrayに型を変換
e_1 = np.array(e_1).T
r_1 = np.corrcoef(e_0,e_1)
r_1

array([[ 1.       ,  0.3395248],
       [ 0.3395248,  1.       ]])

相関係数$corr(e_0,e_1)$は、1.の計算結果と一致していることが分かる。

ちなみに、変数を中心化して行った回帰と通常の回帰で、両者の残差は一致する(教科書p.210)。  
$\hat{y} = \hat{{\theta_{0}}}+{\hat{\theta_{2}}}x_{2}, \hspace{15pt} e_0=y - {\hat{y}} = y_c - \hat{\hat{y_c}}$  

実際のデータで一致しているか確かめてみる。

In [19]:
#統計ライブラリを使用
import statsmodels.api as sm
X_ = np.column_stack((np.repeat(1,n),x_2)) #切片項を追加 
model = sm.OLS(y,X_) # y = theta_0 + theta_2 * x_2 + eps
results = model.fit()
print("通常の回帰の残差")
print(results.resid)
print("変数を中心化した回帰の残差")
print(e_0[0])

通常の回帰の残差
[-0.54313725  5.81960784 -2.21960784 -0.89607843  2.85882353 -2.66078431
  0.81960784  2.22156863 -4.66078431 -0.73921569]
変数を中心化した回帰の残差
[-0.54313725  5.81960784 -2.21960784 -0.89607843  2.85882353 -2.66078431
  0.81960784  2.22156863 -4.66078431 -0.73921569]


###3. 単相関係数を用いる方法  

コントロールする変数（影響を取り除く変数）が一つの場合は、
偏相関係数を各変数間の相関係数を用いて表すことができる。

$Corr(e_0,e_1) = \frac{Cov(e_0,e_1)}{\sqrt{Var(e_0)}\sqrt{Var(e_1)}}$

ここで、$Cov(e_0,e_1) = (1/n)y_c'[I-x_{2c}(x_{2c}'x_{2c})^{-1}x_{2c}']x_{1c}$、　$([I-x_{2c}(x_{2c}'x_{2c})^{-1}x_{2c}']$は対称、べき等)

$\sqrt{Var(e_0)} = \sqrt{(1/n)y_c'[I-x_{2c}(x_{2c}'x_{2c})^{-1}x_{2c}']y_c}$、

$\sqrt{Var(e_1)} =　\sqrt{(1/n)x_{1c}'[I-x_{2c}(x_{2c}'x_{2c})^{-1}x_{2c}']x_{1c}}$ だから、

1.で用いた共分散を示す記号$s_{ij}$を用いると、

$Corr(e_0,e_1)=\frac{s_{01}-s_{02}s_{12x}s_{22}^{-1}}{\sqrt{s_{00}-s_{02}^2s_{22}^{-1}}\sqrt{s_{11}-s_{12}^2s_{22}^{-1}}}$
$\hspace{15pt}$ となる。
(コントロール変数が1つなので、それとの共分散の部分はスカラー)

単相関係数$r_{ij}=s_{ij}/\sqrt{s_{ii}}\sqrt{s_{jj}}$を用いた表示では、

$Corr(e_0,e_1)=\frac{r_{01}-r_{02}r_{12}}{\sqrt{1-r_{02}^{2}}\sqrt{1-r_{12}^{2}}}$　となる。

全変数についての相関係数の行列を求める。  
$A = [y,x_1,x_2]$とし、$A$を各列について標準化したものを改めて$Z$とおく。

In [20]:
A = np.column_stack((y,x_1,x_2))
Z = (A - A.mean(axis=0)) / A.std(axis=0) #列ごとに標準化
Z

matrix([[-1.70612028, -1.78532808, -1.77122977],
        [ 0.57766277,  0.08638684, -0.22140372],
        [-0.09403813,  0.23036491,  0.22140372],
        [ 1.92106457,  0.51832106,  2.21403721],
        [-0.22837831, -0.63350351, -0.66421116],
        [-0.76573902, -0.77748158, -0.44280744],
        [-0.09403813,  1.52616755, -0.22140372],
        [ 1.11502349,  1.67014562,  0.88561489],
        [-1.03441938, -0.34554737, -0.44280744],
        [ 0.30898241, -0.48952544,  0.44280744]])

相関係数行列は、$(1/n)Z'Z$で表せるから、

In [21]:
ZZ = (1/n)*Z.T * Z
ZZ

matrix([[ 1.        ,  0.67348983,  0.92204589],
        [ 0.67348983,  1.        ,  0.61841925],
        [ 0.92204589,  0.61841925,  1.        ]])

相関行列の要素を用いて、$y$と$x_1$の偏相関係数を計算。

In [22]:
(ZZ[0,1] - ZZ[0,2]*ZZ[1,2]) / (math.sqrt(1-pow(ZZ[0,2],2)) * math.sqrt(1-pow(ZZ[1,2],2)))

0.33952479611309688

1.、2.で求めた値と一致することが確かめられた。